# Top Pages (Last 7 Days)

This notebook queries GA4 export data in BigQuery and shows which pages received the most traffic in the last 7 days.

**Project:** `lifeline-website-480522`  
**Dataset:** `analytics_315584957`

In [1]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px

PROJECT_ID = "lifeline-website-480522"
DATASET = "analytics_315584957"
TOP_N = 15

client = bigquery.Client(project=PROJECT_ID)

In [2]:
query = f"""
SELECT
  COALESCE((
    SELECT ep.value.string_value
    FROM UNNEST(event_params) ep
    WHERE ep.key = 'page_location'
  ), '(unknown)') AS page_location,
  COUNT(*) AS page_views
FROM `{PROJECT_ID}.{DATASET}.events_*`
WHERE event_name = 'page_view'
  AND _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY))
  AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
GROUP BY page_location
ORDER BY page_views DESC
LIMIT {TOP_N}
"""

job = client.query(query)
df = job.to_dataframe()
df

/home/aido/projects/lla-data/.venv/lib/python3.14/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,page_location,page_views
0,https://www.lifeline.org.au/,12813
1,https://www.lifeline.org.au/get-help/national-...,4898
2,https://www.lifeline.org.au/chat?q=crisis-chat/,3875
3,https://www.outoftheshadowswalk.org.au/tribute...,3539
4,https://www.lifeline.org.au/chat,3509
5,https://www.lifeline.org.au/search?search=depr...,3401
6,https://www.lifeline.org.au/get-help/support-t...,2168
7,https://www.lifeline.org.au/chat?q=get-help/se...,2141
8,https://www.lifeline.org.au/chat/staging-chat,2018
9,https://www.lifeline.org.au/131114?q=get-help/...,942


In [3]:
fig = px.bar(
    df,
    x="page_location",
    y="page_views",
    title="Top Pages by Page Views (Last 7 Days)",
    labels={"page_location": "Page URL", "page_views": "Page Views"},
)
fig.update_layout(xaxis_tickangle=-45, height=650)
fig.show()

If you get an auth error, run this once in terminal:

```bash
gcloud auth application-default login
```